In [2]:
%matplotlib notebook
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
df = pd.read_csv("G:\\XX\\Data\\boston.csv", header=0)
print(df.describe())

             CRIM         ZN       INDUS         CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677082   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         RAD         TAX     PTRATIO       LSTAT  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534   12.653063   
std     28.148861    2.1057

In [3]:
# 把df转换为np的数组格式
df = np.array(df)
# 特征数据归一化处理0-1
for i in range(12):
    df[:, i]=df[:, i]/(df[:, i].max()-df[:, i].min())
# x_data为前12列特征数据
x_data = df[:, :12]
# y_data为最后一列标签数据
y_data = df[:, 12]
# print(x_data, x_data.shape)
# print(y_data, y_data.shape)
x=tf.placeholder(tf.float32, [None, 12], name="X")
y=tf.placeholder(tf.float32, [None, 1], name="Y")

In [4]:
# 定义一个命名空间
with tf.name_scope("Model"):
    w=tf.Variable(tf.random_normal([12,1], stddev=0.01), name="W")
    b=tf.Variable(1.0, name="b")
    def model(x,w,b):
        return tf.matmul(x,w)+b
    pred=model(x,w,b)

In [5]:
# 设置训练超参数
train_epochs=200
learning_rate=0.01
# 定义损失函数:均方误差
with tf.name_scope("LossFunction"):
    loss_function = tf.reduce_mean(tf.pow(y-pred, 2))
# 创建优化器
optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

In [6]:
# 为Tensorboard可视化准备数据
logdir='G:\XX\JupyterNotebook\log'
sum_loss_op=tf.summary.scalar("loss", loss_function)
merged=tf.summary.merge_all()

In [ ]:
# 模型训练
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)
loss_list=[]

# 创建摘要Writer，将计算图写入摘要文件，后面再Tensorboard中Graphs栏可见
writer=tf.summary.FileWriter(logdir, sess.graph)

for epoch in range(train_epochs):
    loss_sum=0.0
    for xs, ys in zip(x_data, y_data):
        # 数据变形，要和placeholder的shape保持一致
        xs=xs.reshape(1,12)
        ys=ys.reshape(1,1)
        
        _, summary_str, loss=sess.run([optimizer, sum_loss_op, loss_function], feed_dict={x: xs, y: ys})
        writer.add_summary(summary_str, epoch)
        
        loss_sum = loss_sum+loss
    # 打乱数据顺序
    xvalues, yvalues=shuffle(x_data, y_data)
    
    b0temp=b.eval(session=sess)
    w0temp=w.eval(session=sess)
    loss_average=loss_sum/len(y_data)
    
    loss_list.append(loss_average)
    print("epoch=", epoch+1, "loss=",loss_average, "b=",b0temp, "w=", w0temp)

epoch= 1 loss= 44.349322221773924 b= 3.6112819 w= [[-0.6084451 ]
 [ 1.3743279 ]
 [-0.79669213]
 [ 0.4985124 ]
 [ 2.5217052 ]
 [ 7.1594844 ]
 [-0.04841596]
 [ 0.7844514 ]
 [ 0.3817717 ]
 [ 0.3256724 ]
 [ 2.3213444 ]
 [-4.4085736 ]]
epoch= 2 loss= 32.03949688652737 b= 4.0001373 w= [[-1.1584827 ]
 [ 1.9590765 ]
 [-1.5259167 ]
 [ 0.8519908 ]
 [ 2.8956883 ]
 [10.598626  ]
 [-0.8167311 ]
 [ 0.34626314]
 [ 0.6297084 ]
 [-0.2647092 ]
 [ 1.1581362 ]
 [-8.123838  ]]
epoch= 3 loss= 27.340999958946096 b= 4.281636 w= [[ -1.6567258 ]
 [  2.1774647 ]
 [ -1.9749283 ]
 [  1.0611266 ]
 [  3.175154  ]
 [ 13.196059  ]
 [ -1.1937649 ]
 [ -0.18538226]
 [  0.9356842 ]
 [ -0.76517564]
 [  0.22626454]
 [-10.978865  ]]
epoch= 4 loss= 24.67310698277124 b= 4.503376 w= [[ -2.1144123 ]
 [  2.2047694 ]
 [ -2.2507756 ]
 [  1.1556656 ]
 [  3.3577774 ]
 [ 15.187056  ]
 [ -1.3561964 ]
 [ -0.72843134]
 [  1.2594436 ]
 [ -1.1965151 ]
 [ -0.5079346 ]
 [-13.198872  ]]
epoch= 5 loss= 23.10554894946896 b= 4.692478 w= [[ -2.53

epoch= 36 loss= 19.887101954356588 b= 9.93086 w= [[ -8.976371  ]
 [  1.7060051 ]
 [ -1.3767537 ]
 [  0.12291405]
 [ -0.82680655]
 [ 22.869488  ]
 [ -0.7282434 ]
 [ -6.4589953 ]
 [  5.664951  ]
 [ -5.256405  ]
 [ -2.881701  ]
 [-20.701147  ]]
epoch= 37 loss= 19.866461850464525 b= 10.096486 w= [[ -9.070519  ]
 [  1.7179534 ]
 [ -1.3482275 ]
 [  0.11710919]
 [ -0.9288802 ]
 [ 22.873938  ]
 [ -0.7308261 ]
 [ -6.53032   ]
 [  5.7091193 ]
 [ -5.300132  ]
 [ -2.8959873 ]
 [-20.66913   ]]
epoch= 38 loss= 19.846462833434416 b= 10.261151 w= [[ -9.160688  ]
 [  1.7293093 ]
 [ -1.3210999 ]
 [  0.11174157]
 [ -1.0282679 ]
 [ 22.877419  ]
 [ -0.7336676 ]
 [ -6.5998287 ]
 [  5.75106   ]
 [ -5.3424935 ]
 [ -2.9107347 ]
 [-20.637957  ]]
epoch= 39 loss= 19.827083358595615 b= 10.424807 w= [[ -9.247033  ]
 [  1.7400874 ]
 [ -1.2953166 ]
 [  0.10678026]
 [ -1.1250966 ]
 [ 22.879967  ]
 [ -0.73671395]
 [ -6.6675816 ]
 [  5.79091   ]
 [ -5.38355   ]
 [ -2.9258823 ]
 [-20.60764   ]]
epoch= 40 loss= 19.8083012

epoch= 70 loss= 19.429656189408984 b= 14.937149 w= [[-10.684294  ]
 [  1.8870546 ]
 [ -0.9186013 ]
 [  0.05880477]
 [ -3.3112242 ]
 [ 22.665972  ]
 [ -0.82116294]
 [ -8.153479  ]
 [  6.450285  ]
 [ -6.2318983 ]
 [ -3.4409108 ]
 [-20.074352  ]]
epoch= 71 loss= 19.421126057236453 b= 15.064321 w= [[-10.705542  ]
 [  1.8883084 ]
 [ -0.9142422 ]
 [  0.05892779]
 [ -3.3644378 ]
 [ 22.653484  ]
 [ -0.8225279 ]
 [ -8.187557  ]
 [  6.461743  ]
 [ -6.2501655 ]
 [ -3.4566438 ]
 [-20.066769  ]]
epoch= 72 loss= 19.412767541943214 b= 15.190386 w= [[-10.725861  ]
 [  1.8894527 ]
 [ -0.9101242 ]
 [  0.05909392]
 [ -3.416912  ]
 [ 22.640814  ]
 [ -0.8238112 ]
 [ -8.221046  ]
 [  6.4728928 ]
 [ -6.2680545 ]
 [ -3.4722517 ]
 [-20.059576  ]]
epoch= 73 loss= 19.404584524250648 b= 15.315347 w= [[-10.7452965]
 [  1.8904909]
 [ -0.9062382]
 [  0.0593006]
 [ -3.4686723]
 [ 22.628006 ]
 [ -0.8250176]
 [ -8.253943 ]
 [  6.4837446]
 [ -6.285583 ]
 [ -3.48775  ]
 [-20.052753 ]]
epoch= 74 loss= 19.396564869538146 b

epoch= 104 loss= 19.214104608573876 b= 18.68471 w= [[-11.061322  ]
 [  1.8921974 ]
 [ -0.8523356 ]
 [  0.07584839]
 [ -4.7940536 ]
 [ 22.207386  ]
 [ -0.8363322 ]
 [ -9.058272  ]
 [  6.725287  ]
 [ -6.700575  ]
 [ -3.9110172 ]
 [-19.957794  ]]
epoch= 105 loss= 19.209502111053915 b= 18.778473 w= [[-11.06576   ]
 [  1.8917261 ]
 [ -0.851764  ]
 [  0.07651193]
 [ -4.829589  ]
 [ 22.194141  ]
 [ -0.83617795]
 [ -9.078941  ]
 [  6.731065  ]
 [ -6.7109776 ]
 [ -3.9228885 ]
 [-19.956995  ]]
epoch= 106 loss= 19.204969947138867 b= 18.871399 w= [[-11.069995  ]
 [  1.8912431 ]
 [ -0.8512283 ]
 [  0.07717675]
 [ -4.864748  ]
 [ 22.180931  ]
 [ -0.8360058 ]
 [ -9.099365  ]
 [  6.7367597 ]
 [ -6.7212358 ]
 [ -3.9346502 ]
 [-19.956276  ]]
epoch= 107 loss= 19.200515380983674 b= 18.963465 w= [[-11.074006 ]
 [  1.8907439]
 [ -0.8507277]
 [  0.0778416]
 [ -4.8995495]
 [ 22.167831 ]
 [ -0.8358181]
 [ -9.119542 ]
 [  6.7423706]
 [ -6.7313657]
 [ -3.9463184]
 [-19.955643 ]]
epoch= 108 loss= 19.1961294578617

epoch= 138 loss= 19.091231070614423 b= 21.437372 w= [[-11.13473   ]
 [  1.8721664 ]
 [ -0.8447624 ]
 [  0.09747808]
 [ -5.822021  ]
 [ 21.799694  ]
 [ -0.82546794]
 [ -9.643361  ]
 [  6.884885  ]
 [ -6.99253   ]
 [ -4.2606    ]
 [-19.957243  ]]
epoch= 139 loss= 19.088462308689156 b= 21.506052 w= [[-11.135418  ]
 [  1.8715484 ]
 [ -0.84473604]
 [  0.09805685]
 [ -5.8473797 ]
 [ 21.789152  ]
 [ -0.82507145]
 [ -9.657524  ]
 [  6.888678  ]
 [ -6.999561  ]
 [ -4.2693415 ]
 [-19.957674  ]]
epoch= 140 loss= 19.08573971544161 b= 21.574112 w= [[-11.136062  ]
 [  1.8709307 ]
 [ -0.84471554]
 [  0.09863294]
 [ -5.8725047 ]
 [ 21.77872   ]
 [ -0.82467747]
 [ -9.671548  ]
 [  6.8924336 ]
 [ -7.0065207 ]
 [ -4.2780147 ]
 [-19.958117  ]]
epoch= 141 loss= 19.083048770650493 b= 21.64154 w= [[-11.136667  ]
 [  1.8703153 ]
 [ -0.84469837]
 [  0.09920374]
 [ -5.897386  ]
 [ 21.768353  ]
 [ -0.8242794 ]
 [ -9.685434  ]
 [  6.8961525 ]
 [ -7.013411  ]
 [ -4.286598  ]
 [-19.958565  ]]
epoch= 142 loss= 19.08

epoch= 172 loss= 19.015568790575358 b= 23.452156 w= [[-11.141906  ]
 [  1.8526717 ]
 [ -0.8453886 ]
 [  0.11485825]
 [ -6.5631757 ]
 [ 21.487078  ]
 [ -0.8124916 ]
 [-10.054188  ]
 [  6.9944987 ]
 [ -7.196331  ]
 [ -4.5174093 ]
 [-19.974483  ]]
epoch= 173 loss= 19.013822940347804 b= 23.50239 w= [[-11.141826  ]
 [  1.8521615 ]
 [ -0.84543085]
 [  0.11529783]
 [ -6.581605  ]
 [ 21.479235  ]
 [ -0.81214404]
 [-10.064361  ]
 [  6.997201  ]
 [ -7.201367  ]
 [ -4.523822  ]
 [-19.975002  ]]
epoch= 174 loss= 19.012100517211913 b= 23.55215 w= [[-11.1417265 ]
 [  1.851655  ]
 [ -0.8454708 ]
 [  0.11573358]
 [ -6.5998607 ]
 [ 21.471453  ]
 [ -0.811797  ]
 [-10.074413  ]
 [  6.9998794 ]
 [ -7.206355  ]
 [ -4.5301704 ]
 [-19.975546  ]]
epoch= 175 loss= 19.010400888888917 b= 23.601477 w= [[-11.141624  ]
 [  1.8511509 ]
 [ -0.8455117 ]
 [  0.11616489]
 [ -6.6179523 ]
 [ 21.463743  ]
 [ -0.81145483]
 [-10.084385  ]
 [  7.0025234 ]
 [ -7.2113028 ]
 [ -4.536465  ]
 [-19.97607   ]]
epoch= 176 loss= 19.00

In [ ]:
 plt.plot(loss_list)